In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
dataset_dir = 'CNN_Dataset'

In [4]:
# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

In [5]:
# Data Augmentation and Rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)  # Split training data for validation , change this to 0.3 and check

In [6]:
# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    dataset_dir,  # Path to the folder that contains 'edible' and 'poisonous' folders
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification: edible vs poisonous
    subset='training')  # Use the training subset

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,  # Same folder as training, just use the validation subset
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')  # Use the validation subset

Found 1354 images belonging to 3 classes.
Found 579 images belonging to 3 classes.


In [7]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(4, 4)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(4, 4)),
    
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(4, 4)),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to avoid overfitting
    
    Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

In [9]:
# we can add layers
# add strides to the conv2d layers


In [10]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=50  # You can adjust this number based on your needs, set this to 50 and check
)

Epoch 1/50


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


42/42 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.5308 - loss: 0.6972 - val_accuracy: 0.5486 - val_loss: 0.6904
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5938 - loss: 0.6862 - val_accuracy: 0.6667 - val_loss: 0.6812
Epoch 3/50


2024-09-12 12:14:46.246999: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-09-12 12:14:46.411487: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


42/42 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.5345 - loss: 0.6927 - val_accuracy: 0.5486 - val_loss: 0.6880
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5000 - loss: 0.7038 - val_accuracy: 0.6667 - val_loss: 0.6689
Epoch 5/50


...

42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5529 - loss: 0.6951 - val_accuracy: 0.5486 - val_loss: 0.6906
Epoch 10/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.6667 - val_loss: 0.6836
Epoch 11/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5434 - loss: 0.6906 - val_accuracy: 0.5503 - val_loss: 0.6876
Epoch 12/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9000 - loss: 0.6398 - val_accuracy: 0.3333 - val_loss: 0.7258
Epoch 13/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5424 - loss: 0.6884 - val_accuracy: 0.5486 - val_loss: 0.6879
Epoch 14/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5625 - loss: 0.6816 - val_accuracy: 0.6667 - val_loss: 0.6649
Epoch 15/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5474 - loss: 0.6892 - val_accuracy: 0.5486 - val_loss: 0.6872
Epoch 16/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4062 - loss: 0.7143 - val_accuracy: 0.6667 - val_loss: 0.6

2024-09-12 12:21:59.778846: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


42/42 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.5490 - loss: 0.6880 - val_accuracy: 0.5486 - val_loss: 0.6876
Epoch 18/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6562 - loss: 0.6686 - val_accuracy: 0.6667 - val_loss: 0.6640
Epoch 19/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5529 - loss: 0.6882 - val_accuracy: 0.5625 - val_loss: 0.6865
Epoch 20/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4688 - loss: 0.7006 - val_accuracy: 0.6667 - val_loss: 0.6690
Epoch 21/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5297 - loss: 0.6910 - val_accuracy: 0.5260 - val_loss: 0.6894
Epoch 22/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5625 - loss: 0.6823 - val_accuracy: 0.6667 - val_loss: 0.6605
Epoch 23/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5504 - loss: 0.6905 - val_accuracy: 0.5833 - val_loss: 0.6820
Epoch 24/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5000 - loss: 0.7125 - val_accuracy: 0.6667 - val_loss: 0.6

2024-09-12 12:30:20.049993: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


42/42 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.5644 - loss: 0.6802 - val_accuracy: 0.5573 - val_loss: 0.6822
Epoch 34/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5312 - loss: 0.6699 - val_accuracy: 0.3333 - val_loss: 0.7473
Epoch 35/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.5753 - loss: 0.6795 - val_accuracy: 0.5903 - val_loss: 0.6794
Epoch 36/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5000 - loss: 0.7041 - val_accuracy: 1.0000 - val_loss: 0.5434
Epoch 37/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.5642 - loss: 0.6765 - val_accuracy: 0.6024 - val_loss: 0.6797
Epoch 38/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6250 - loss: 0.6982 - val_accuracy: 0.6667 - val_loss: 0.6234
Epoch 39/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.6044 - loss: 0.6649 - val_accuracy: 0.5799 - val_loss: 0.6783
Epoch 40/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5000 - loss: 0.7285 - val_accuracy: 0.6667 - val_loss: 0.4

In [11]:
# Save the model after training
model.save('CNN_model.h5')

# Evaluate the model on validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")

19/19 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5225 - loss: 0.6968
Validation Accuracy: 51.30%


### Testing the model

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

In [13]:
# Load the trained model
model = tf.keras.models.load_model('CNN_model.h5')

In [14]:
def predict_image_with_percentage(img_path):
    # Load the image
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))  # Use the same image size as during training

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Add a batch dimension (model expects a batch of images)
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize the image (same normalization used during training)
    img_array /= 255.0

    # Predict using the model
    prediction = model.predict(img_array)[0][0]

    # Calculate percentage probabilities
    poisonous_prob = prediction * 100  # Model output is the probability of being poisonous
    edible_prob = (1 - prediction) * 100  # The complement is the probability of being edible

    # Print the probabilities
    print(f"Edible Probability: {edible_prob:.2f}%")
    print(f"Poisonous Probability: {poisonous_prob:.2f}%")

    # Interpret the results
    if poisonous_prob > 50:
        print("The mushroom is likely Poisonous!")
    else:
        print("The mushroom is likely Edible!")

In [22]:
# Load test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    dataset_dir,  # Replace with the path to your test data
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  # Binary classification (edible vs poisonous)
    shuffle=False  # Disable shuffle to match predictions with true labels
)

Found 1933 images belonging to 3 classes.


In [27]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Predict the class probabilities
y_pred = model.predict(test_generator)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)  # Convert probabilities to binary labels

61/61 ━━━━━━━━━━━━━━━━━━━━ 57s 916ms/step


In [29]:
# Get true labels from the test generator
y_true = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

In [30]:
# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[348 712   0]
 [171 700   0]
 [  1   1   0]]


In [31]:
# Optional: Print a classification report (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=['Edible', 'Poisonous']))

# Plot confusion matrix as a heatmap for better visualization
plt.figure(figsize=(6, 4))
sns.heatmap(
    cm,
    annot = True,
    fmt = "d",
    cmap = 'Blues',
    xticklabels=['Edible', 'Poisonous'],
    yticklabels=['Edible', 'Poisonous']
)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


Classification Report:


ValueError: Number of classes, 3, does not match size of target_names, 2. Try specifying the labels parameter

In [15]:
# Edible Image
predict_image_with_percentage('CNN_Dataset/Edible/1666181903_coprinus-comatus-3.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Edible Probability: 43.74%
Poisonous Probability: 56.26%
The mushroom is likely Poisonous!


In [16]:
# Edible Image

predict_image_with_percentage('CNN_Dataset/Edible/hericium-americanum-1.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Edible Probability: 17.85%
Poisonous Probability: 82.15%
The mushroom is likely Poisonous!


In [17]:
# Poisonous Image

predict_image_with_percentage('CNN_Dataset/Poisonous/geastrum-triplex-1.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Edible Probability: 39.35%
Poisonous Probability: 60.65%
The mushroom is likely Poisonous!


In [18]:
# Poisonous Image

predict_image_with_percentage('CNN_Dataset/Poisonous/clathrus-archeri-5.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Edible Probability: 5.95%
Poisonous Probability: 94.05%
The mushroom is likely Poisonous!


In [19]:
# Test Poisonous Image

predict_image_with_percentage('CNN_Dataset/Test/Poisonous.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Edible Probability: 31.38%
Poisonous Probability: 68.62%
The mushroom is likely Poisonous!


In [20]:
# Test Edible Image

predict_image_with_percentage('CNN_Dataset/Test/Edible.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Edible Probability: 63.20%
Poisonous Probability: 36.80%
The mushroom is likely Edible!
